In [1]:
# import all libraries
import torch
import torch.nn as nn
from torch.utils.data import random_split
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms

import os
import argparse


In [ ]:
import pickle

# Define transforms (keep these - they're still needed!)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the saved datasets from pickle files
print("Loading datasets from pickle files...")

with open('./data/train_dataset.pkl', 'rb') as f:
    trainset = pickle.load(f)

with open('./data/val_dataset.pkl', 'rb') as f:
    valset = pickle.load(f)

# Don't load test for Section 4 (not needed)
# with open('./data/test_dataset.pkl', 'rb') as f:
#     testset = pickle.load(f)

print(f"✅ Loaded! Train={len(trainset)}, Val={len(valset)}")

# Create DataLoaders
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(
    valset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')



---



In [ ]:
# Training
def train(epoch, net, criterion, trainloader, scheduler=None):
    device = 'cuda'
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (batch_idx+1) % 50 == 0:
          print("iteration : %3d, loss : %0.4f, accuracy : %2.2f" % (batch_idx+1, train_loss/(batch_idx+1), 100.*correct/total))

    if scheduler is not None:
        scheduler.step()
    return train_loss/(batch_idx+1), 100.*correct/total

Test performance on the test set. Note the use of `torch.inference_mode()`

In [ ]:
def test(epoch, net, criterion, testloader):
    device = 'cuda'
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.inference_mode():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss/(batch_idx+1), 100.*correct/total



In [ ]:
def save_checkpoint(net, acc, epoch):
    # Save checkpoint.
    print('Saving..')
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.pth')





In [ ]:
class CustomBatchNorm2d(nn.Module):
    """
    Custom Batch Normalization where mean and std do NOT participate
    in gradient calculation using detach().
    """
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(CustomBatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        # Learnable parameters (gamma and beta)
        self.weight = nn.Parameter(torch.ones(num_features))
        self.bias = nn.Parameter(torch.zeros(num_features))

        # Running statistics for inference (not learnable)
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))

    def forward(self, x):
        # Input shape: (N, C, H, W)
        # N = batch size, C = channels, H = height, W = width

        if self.training:
            # Calculate batch statistics across (N, H, W), keep C dimension
            mean = x.mean(dim=[0, 2, 3], keepdim=False)  # Shape: (C,)
            var = x.var(dim=[0, 2, 3], keepdim=False, unbiased=False)  # Shape: (C,)

            # CRITICAL: Detach mean and var from computation graph
            # This prevents gradients from flowing through these statistics
            mean_detached = mean.detach()
            var_detached = var.detach()

            # Update running statistics (no gradients needed here)
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + \
                                   self.momentum * mean_detached
                self.running_var = (1 - self.momentum) * self.running_var + \
                                  self.momentum * var_detached
                self.num_batches_tracked += 1

            # Normalize using DETACHED statistics
            # Reshape for broadcasting: (1, C, 1, 1)
            x_normalized = (x - mean_detached.view(1, -1, 1, 1)) / \
                          torch.sqrt(var_detached.view(1, -1, 1, 1) + self.eps)

        else:
            # Inference mode: use running statistics
            x_normalized = (x - self.running_mean.view(1, -1, 1, 1)) / \
                          torch.sqrt(self.running_var.view(1, -1, 1, 1) + self.eps)

        # Scale and shift using learnable parameters
        # weight (gamma) and bias (beta) WILL have gradients
        out = self.weight.view(1, -1, 1, 1) * x_normalized + \
              self.bias.view(1, -1, 1, 1)

        return out

    def extra_repr(self):
        return f'{self.num_features}, eps={self.eps}, momentum={self.momentum}'

In [ ]:
# defining resnet models

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = CustomBatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = CustomBatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                CustomBatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # This is the "stem"
        # For CIFAR (32x32 images), it does not perform downsampling
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = CustomBatchNorm2d(64)
        # four stages with three downsampling
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test_resnet18():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


#Hyperparameter Tuning

## Batch Normalization

Chosen LR from prev experiment = 0.01

Chosen Weight decay =  1e-2

Epochs = 300

With Cosine Annealing

Other hyperparameters unchanged

In [ ]:
chosen_LR = 0.01
chosen_WD =  1e-2

# main body
config = {
    'lr': chosen_LR,
    'momentum': 0.9,
    'weight_decay': chosen_WD
}

In [ ]:
# Section 6: Custom Batch Normalization

print(f"\n{'='*60}")
print(f"Section 6: Custom Batch Normalization")
print(f"{'='*60}\n")

# Create model with CustomBatchNorm2d
net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'],weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

# Lists to store losses and accuracies
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(1, 301):
    train_loss, train_acc = train(epoch, net, criterion, trainloader, scheduler)
    val_loss, val_acc = test(epoch, net, criterion, valloader)

    # Store values
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, val loss " + \
          ": %0.4f, val accuracy : %2.2f") % (epoch, train_loss, train_acc, val_loss, val_acc))

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(range(1, 301), train_losses, 'b-', label='Train', linewidth=2)
ax1.plot(range(1, 301), val_losses, 'r-', label='Validation', linewidth=2)
ax1.set_xlabel("Epoch", fontsize=12)
ax1.set_ylabel("Loss", fontsize=12)
ax1.set_title("Loss vs Epochs (Custom BatchNorm)", fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

ax2.plot(range(1, 301), train_accs, 'b-', label='Train', linewidth=2)
ax2.plot(range(1, 301), val_accs, 'r-', label='Validation', linewidth=2)
ax2.set_xlabel("Epoch", fontsize=12)
ax2.set_ylabel("Accuracy (%)", fontsize=12)
ax2.set_title("Accuracy vs Epochs (Custom BatchNorm)", fontsize=14)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('section6_custom_batchnorm.png', dpi=300, bbox_inches='tight')
plt.show()

# Save the model
torch.save(net.state_dict(), 'section6_best_model.pth')

print(f"\n{'='*60}")
print(f"Final Results - Custom BatchNorm:")
print(f"  Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.2f}%")
print(f"  Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accs[-1]:.2f}%")
print(f"{'='*60}\n")